In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress


In [ ]:
# find and read the data file, api or csv? 
# using watson_healthcare_modified.csv


In [ ]:
# pull/sort the data needed;
employee_id = [] # to verify no duplicates
attrition = [] # wether still working or not
job_role = [] # be sure we are using only nurses
monthly_income = [] #use this as base pay
percent_salary_hike = [] # us for recent pay raises
business_travel = [] # travel_rarely considered full-time, travel_frequently considered part-time or travel nurses



In [ ]:
# setup dataframe with data retrieved


In [ ]:
# analyze data with plots of the following;
# pie chart - ft vs pt
# bar chart - attrition vs type (ft and pt)
# bar chart - salary vs attrition vs type (ft / pt)

